In [21]:
%cd ..
from llm_lora import QwenQACInference, load_model_qwen
from speedy import *
from sklearn.metrics import classification_report, f1_score
from guidance import *

def pretty_print(squad_data):
    with system():
        lm = llm + squad_data.get('system', 'You are a helpful assistant.')
    with user():
        lm += squad_data['conversations'][0]['value']
    with assistant():
        lm += squad_data['conversations'][1]['value']
    return lm

def evaluate(outputs, labels):
    best_thr = 0
    for thr in np.arange(0.2, .8, 0.05):
        preds = [1 if score > thr else 0 for score in outputs]
        score = f1_score(labels, preds)
        best_thr = max(best_thr, score)
        print('F1 score at threshold {} is {}'.format(thr, score))
    print('Best F1 score is {}'.format(best_thr))    

/anhvth5


In [22]:
model, tok = load_model_qwen(
    '/anhvth5/checkpoint-llm-finetuned/qwen_14B/full_010124/', device_map=0, bf16=True)
qac = QwenQACInference(model=model, tokenizer=tok)
print('QwenQACInference loaded')
llm = models.QwenChat(model, tok)

2024-01-06 11:50:08.093 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


QwenQACInference loaded


In [3]:
# check dtype model param, get the first param
print('Model param dtype:', next(model.parameters()).dtype)

Model param dtype: torch.bfloat16


In [5]:
# ls ../../data/chat-formated-dataset/

In [15]:
nli_data = load_by_ext(
    '../../data/chat-formated-dataset/tvpl_giaothong_rag_nli_vietnamese_train.json')

In [16]:

# Example:  {'conversations': [{'from': 'user', 'value': 'Please determine the relationship between the following premise and hypothesis:\nPremise: ```“Đi bão” là hành động rất đông người dân đổ xô ra đường phố cùng nhau ăn mừng chiến thắng, thường xuất hiện trong những sự kiện bóng đá có sự tham gia của các Đội tuyển bóng đá nam Việt Nam. Hoạt động này không có định nghĩa, khái niệm trong các văn bản quy phạm pháp luật. Tuy nhiên, nếu đi bão gây混乱 giao thông, gây tổn hại đến tài sản công hoặc của cá nhân, người tham gia có thể bị xử lý theo quy định của pháp luật.```\n\nHypothesis: ```Văn bản hợp nhất 15/VBHN-VPQH 2019 Luật Giao thông đường bộ\nChương I |  Điều 3. Giải thích từ ngữ\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n10. Dải phân cách là bộ phận của đường để phân chia mặt đường thành hai chiều xe chạy riêng biệt hoặc để phân chia phần đường của xe cơ giới và xe thô sơ. Dải phân cách gồm loại cố định và loại di động.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n11. Nơi đường giao nhau cùng mức (sau đây gọi là nơi đường giao nhau) là nơi hai hay nhiều đường bộ gặp nhau trên cùng một mặt phẳng, gồm cả mặt bằng hình thành vị trí giao nhau đó.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n12. Đường cao tốc là đường dành cho xe cơ giới, có dải phân cách chia đường cho xe chạy hai chiều riêng biệt; không giao nhau cùng mức với một hoặc các đường khác; được bố trí đầy đủ trang thiết bị phục vụ, bảo đảm giao thông liên tục, an toàn, rút ngắn thời gian hành trình và chỉ cho xe ra, vào ở những điểm nhất định.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n13. Đường chính là đường bảo đảm giao thông chủ yếu trong khu vực.\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n14. Đường nhánh là đường nối vào đường chính.```'}, {'from': 'assistant', 'value': '4. Neutral/Irrelevant'}]}
def pretty_print_nli_data(nli_data):
    for conv in nli_data['conversations']:
        print(conv['from'], '>>>', conv['value'])


pretty_print(nli_data[10])

In [17]:
qac_test = load_by_ext('/anhvth5/data/qac-data/za_qac/qac_test.jsonl')
qac_test[0]

{'id': 'test_1001p1',
 'question': 'Con trai đầu của Ngô Quyền tên gì',
 'title': 'Nhà Ngô',
 'text': 'Ngô Quyền hạ thành Đại La , giết Công Tiễn rồi bày trận trên sông Bạch Đằng đón quân Nam Hán .',
 'label': False}

## Forward batches

In [19]:

documents = ['{} | {}'.format(item['title'], item['text'])
            for item in qac_test]
questions = [item['question'] for item in qac_test]
outputs = qac.forward(questions, documents,
                    template='5level', return_scores=True)


100%|██████████| 110/110 [00:37<00:00,  2.91it/s]


In [24]:
labels = [item['label'] for item in qac_test]
evaluate(outputs, labels)

F1 score at threshold 0.2 is 0.4790610829554548
F1 score at threshold 0.25 is 0.6556169429097607
F1 score at threshold 0.3 is 0.8158878504672898
F1 score at threshold 0.35 is 0.8454011741682974
F1 score at threshold 0.39999999999999997 is 0.8657921291624621
F1 score at threshold 0.44999999999999996 is 0.8736462093862816
F1 score at threshold 0.49999999999999994 is 0.8812301166489926
F1 score at threshold 0.5499999999999999 is 0.8833423765599565
F1 score at threshold 0.5999999999999999 is 0.8845510317902956
F1 score at threshold 0.6499999999999999 is 0.8748564867967853
F1 score at threshold 0.7 is 0.8684365781710914
F1 score at threshold 0.7499999999999998 is 0.8550548112058465
F1 score at threshold 0.7999999999999998 is 0.8432229856339787
Best F1 score is 0.8845510317902956


# Use NLI as QAC

In [25]:
squad_data = load_by_ext(
    '/anhvth5/data/chat-data/za_chat/envi_squad_train.json'
)

In [37]:
lm = pretty_print(squad_data[100])

In [60]:
item = qac_test[13]
question = item['question']
document = '{} | {}'.format(item['title'], item['text'])
# answer = item['answer']
answer = 'thế kỷ XVI'
question = '<blank'
print('Question:', question)
print('Document:', document)
user_data = {"document": document, "question": question, "answer": answer, "language": "vi"}
user_data = json.dumps(user_data, ensure_ascii=False, indent=4 )
res = model.chat(tok, user_data, system='Task: Field in the blank', history=[], do_sample=True)[0]
print('>>>', res)

Question: <blank
Document: Nhà Lê sơ | Thời đại Lê sơ có 10 vị Hoàng đế thuộc 6 thế hệ , đây là thời kỳ các Hoàng đế nhà Lê nắm trọn được quyền hành , cũng là thời kỳ vĩ đại , hoàng kim của chế độ phong kiến Việt Nam . Đời Lê Thái Tổ ( Lê Lợi ) và đặc biệt là đời Lê Thái Tông , xã hội được đi vào ổn định , phát triển thịnh vượng một cách mau chóng sau thời kỳ chiến tranh trước đó .
>>> question: Thời gian trị vì của triều đại nhà Hậu Lê là lúc nào ?


In [107]:

# question = qac.generate_question(item['text'], '10 vị vua')
qac.generate_answer(question, item['text'])

'10'

In [126]:
qac.forward_batch(
    ['Việc thông báo trục vớt tài sản chìm đắm gây nguy hiểm được thực hiện trong thời gian nào? '],
    ['''Nghị định 05/2017/NĐ-CP xử lý tài sản chìm đắm đường thủy nội địa vùng nước cảng biển vùng biển Việt ...
Chương II |  Điều 13. Thủ tục phê duyệt phương án trục vớt tài sản chìm đắm
3. Trường hợp hồ sơ nhận được không hợp lệ thì chậm nhất 02 ngày làm việc kể từ ngày nhận được hồ sơ, cơ quan có thẩm quyền phê duyệt phải hướng dẫn người nộp hoàn thiện hồ sơ theo quy định tại Nghị định này. Nếu hồ sơ hợp lệ thì trong thời hạn không quá 10 ngày kể từ ngày nhận được hồ sơ, cơ quan có thẩm quyền phê duyệt phải có văn bản phê duyệt phương án trục vớt tài sản chìm đắm và gửi trực tiếp đến người nộp hồ sơ hoặc gửi qua hệ thống bưu chính; trường hợp không phê duyệt phải có văn bản trả lời và nêu rõ lý do.
4. Đối với tài sản chìm đắm gây nguy hiểm, cơ quan có thẩm quyền phải tổ chức phê duyệt phương án trục vớt trong thời hạn không quá 24 giờ kể từ thời điểm nhận được hồ sơ hợp lệ; đối với trường hợp quy định tại điểm a khoản 4 Điều 12 thì thời hạn này không quá 48 giờ.'''],
'nli', return_scores=True, expected_answers='Theo Điều 279 Bộ luật Hàng hải Việt Nam 2015, chủ sở hữu tài sản chìm đắm gây nguy hiểm có nghĩa vụ thông báo ngay cho Giám đốc Cảng vụ hàng hải nơi gần nhất biết sự cố xảy ra và phải tiến hành trục vớt tài sản đó trong thời hạn theo quy định của Chính phủ.')

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
          151645,    198, 151644,    872,    198,   5501,   8253,    279,   5025,
            1948,    279,   2701,  40202,    323,  30078,    510,  41462,   1064,
              25,  54275,  14582,     25, 133817,  92430, 128603,    489,  92704,
             348, 132540, 128556,  85636,    521, 126303,  14854, 125238, 128882,
            7777,   4076, 131491,  63478, 128336, 128283,  69086, 128327,    342,
            1103, 128352,     30,    715,  16141,     25,    350,  13874,  19324,
              39,  59300,  13189,     25,  54275,     45,    866,  35928, 128333,
             220,     15,     20,     14,     17,     15,     16,     22,  20290,
           40225,     12,   7123, 129888, 128356, 128556,  85636,    521, 126303,
           14854, 125238, 128448, 132601, 128811, 128704, 129838, 128301,    272,
          124428, 130027, 129838, 130027, 128324,  12236,   1143,  85375,   7946,
   

In [166]:
pretty_print(nli_data[32])

In [144]:
eval(qac.query_parsing('trong cơ thể có ma tuý có được đi ô tô'))

{'query_type': 'tra_cuu_quy_dinh',
 'violation': 'có ma túy trong cơ thể',
 'object': 'khong_ro',
 'specific_inquiry': 'điều kiện để lái xe ô tô khi có ma túy trong cơ thể'}